In [ ]:
from __future__ import print_function
import fitz, sys

def split_pdf(finput):
    doc = fitz.open(finput)
    page = doc.loadPage(0)
    if int(page.MediaBox[2]) > 800.0:
        res = fitz.open() 

        for spage in doc:
            r = spage.rect  
            d = fitz.Rect(spage.CropBoxPosition, spage.CropBoxPosition)  
    
            r1 = fitz.Rect(0, 0, r.width*0.5, r.height)
            r2 = fitz.Rect(r.width*0.5, 0, r.width, r.height)
            rect_list = [r1, r2]
    
            for rx in rect_list:  
                rx += d 
                page = res.newPage(-1, width = rx.width, height = rx.height)
                page.showPDFpage(page.rect, doc, spage.number, clip = rx)

        res.save(doc.name[:-3]+'_sp.pdf', garbage = 3, deflate = True)
        return doc.name[:-3]+'_sp.pdf'
    else:
        return finput

In [ ]:
import pdftotext
import os

def text22text(finput):
    
    fout = os.path.join(os.getcwd(), 'output.txt')
    
    if '.pdf' in finput:
         
        with open(finput, "rb") as f:
            pdf = pdftotext.PDF(f)

        with open('output.txt', 'w',-1,'utf-8') as f:
            f.write("\n\n".join(pdf))
    
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
            
    elif '.HWP' or '.hwp' in finput:
        hwp2txt_cmd = r'hwp5txt --output={} {}'.format(fout, finput)
        os.system(hwp2txt_cmd)
        with open(fout,'r',encoding='UTF8') as f:
            text=f.read()
            
    return text

In [ ]:
import glob
from PIL import Image
from PyPDF2 import PdfFileReader
from pdf2image import convert_from_bytes
from pathos.multiprocessing import ProcessingPool as Pool

def image22text(finput, seqnum):
    img_name = os.path.join(r'C:\pgm_file', 'img_file', seqnum) 
    if '.hwp' in finput:
        return 0
    
    if not os.path.exists(img_name):
        os.mkdir(img_name)
        images = convert_from_bytes(open(finput, 'rb').read(), dpi=300, output_folder = img_name, fmt="png")
    
    img_list = list()
    for i in range(len(os.listdir(img_name))):
        img_list.append(img_name)
        
    with Pool(10) as p:
        text = p.map(image_multi, img_list, os.listdir(img_name))
    return " ".join(text)

def image_multi(img_name, img):
    import cv2
    import os
    import pytesseract
    img = cv2.imread(os.path.join(img_name, img))
    img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)

    custom_config = r'-l kor -c tessedit_char_blacklist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 6'
    temp = pytesseract.image_to_string(thresh, config = custom_config, lang='kor')
    return temp

In [ ]:
import os
import pandas as pd 

tlist, ilist = list(), list()
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
df = pd.read_excel(xname)

for i in range(len(df)):
    
    finput = df.loc[i, 'path']
    print(finput)
    if 'pdf' in finput:
        finput = split_pdf(finput)

    text = text22text(finput)
    if text.isspace():
        text = image22text(finput, str(df.loc[i, 'seqnum']))
        ilist.append('1')
    else:
        ilist.append('0')
    tlist.append(text)
    
    print(i)
df['text'] = tlist
df['image'] = ilist
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df.to_excel(x2name)